# Re-analysis:    
# Single-cell profiling of tumor heterogeneity and the microenvironment in advanced non-small cell lung cancer
https://doi.org/10.1038/s41467-021-22801-0

> **Abstract**: Lung cancer is a highly heterogeneous disease. Cancer cells and cells within the tumor microenvironment together determine disease progression, as well as response to or escape from treatment. To map the cell type-specific transcriptome landscape of cancer cells and their tumor microenvironment in advanced non-small cell lung cancer (NSCLC), we analyze 42 tissue biopsy samples from stage III/IV NSCLC patients by single cell RNA sequencing and present the large scale, single cell resolution profiles of advanced NSCLCs. In addition to cell types described in previous single cell studies of early stage lung cancer, we are able to identify rare cell types in tumors such as follicular dendritic cells and T helper 17 cells. Tumors from different patients display large heterogeneity in cellular composition, chromosomal structure, developmental trajectory, intercellular signaling network and phenotype dominance. Our study also reveals a correlation of tumor heterogeneity with tumor associated neutrophils, which might help to shed light on their function in NSCLC.

In [9]:
# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")
# BiocManager::install('Seurat', dependencies=TRUE)
# BiocManager::install("harmony", dependencies=TRUE)

In [1]:
library(data.table)
library(Seurat)
library(harmony)

Attaching SeuratObject

Loading required package: Rcpp



### Read data
The raw sequencing data were deposited at Gene Expression Omnibus GSE148071. The published data used for validation or comparation in this study were retrieved from the NCBI Gene Expression Omnibus database accession code GSE13190712, GSE992549, and ArrayExpress under Accessions E-MTAB-61498. The remaining data are available within the Article, Supplementary Information or available from the authors upon request.

In [17]:
data_url <- paste(getwd(), .Platform$file.sep, "GSE148071_data", sep="")
files <- list.files(data_url)

In [18]:
### -> test set of files
files <- files[1:3]

In [19]:
tables <- list()
for (f in files) {
    print(paste("Loading", f, "..."))
    table <- read.table(
        file = paste(data_url, f, sep=.Platform$file.sep), 
        sep = "\t", 
        header = TRUE)
    tables <- append(tables, list(table))
}
names(tables) <- files
print("Table loading complete.")

[1] "Loading GSM4453576_P1_exp.txt ..."
[1] "Loading GSM4453577_P2_exp.txt ..."
[1] "Loading GSM4453578_P3_exp.txt ..."
[1] "Table loading complete."


### Clean data
We removed cells that had either lower than 200 or higher than 5000 expressed genes. Furthermore, we discarded cells with more than 30,000 UMIs and mitochondria content higher than 30%.

In [52]:
gene_min <- 200
gene_max <- 5000
umi_max <- 30000
mito_percent <- 0.3

remove_tables <- list()
for (t in names(tables)) {
    zero_table <- tables[[t]][rowSums(tables[[t]][]) > 0, ]
    dim_genes <- dim(zero_table)[1]
    dim_samples <- dim(zero_table)[2]
    if (dim_genes < gene_min || dim_genes > gene_max || dim_samples > umi_max) {
        remove_tables <- append(remove_tables, list(t))
    }
}
print(remove_tables)

[[1]]
[1] "GSM4453576_P1_exp.txt"

[[2]]
[1] "GSM4453577_P2_exp.txt"

[[3]]
[1] "GSM4453578_P3_exp.txt"



In [53]:
t <- 'GSM4453576_P1_exp.txt'

In [60]:
tables[[t]][rowSums(tables[[t]]) > 0, ]

,X1_TAGGCTAAGCCG,X1_CCGGCATATTCG,X1_TCCGTAGACGTA,X1_AAACTACAGTGC,X1_GGTGAAAAGAAT,X1_GGCTAGTGGAAG,X1_TTTTCTCCCGCT,X1_CCATCAACGCCA,X1_GGTTGCATCTAA,X1_TTATTGCCTTAG,...,X2_GCGTGCTTCTCA,X2_AAACTCCGGCTT,X2_TACGGCATCCAC,X2_TACAAGGGCCTG,X2_CTACGCAGTGCT,X2_CGGAGTAATTCC,X2_GGCGGGCGCTGC,X2_GGAACGTAATAC,X2_CGAAAGGACCAG,X2_CAAACAGTATAA
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
PWP1,8,2,3,5,3,0,1,0,7,6,...,0,4,0,1,0,0,1,0,0,1
FTH1,608,521,425,547,348,264,904,603,298,271,...,62,88,1,55,57,56,46,41,70,48
PERP,20,46,30,31,36,41,16,48,30,37,...,0,7,0,11,3,0,14,12,0,1
RPL37,130,195,153,119,76,182,131,120,118,98,...,1,8,2,7,23,10,12,15,5,12
MT-RNR2,839,1515,1309,1122,1048,2000,1305,824,1007,1646,...,194,115,67,115,8,220,141,88,226,120
AKR1C3,26,35,43,28,70,64,27,48,37,30,...,1,3,0,3,10,0,8,10,0,1
ATP6V0E1,5,7,10,2,3,7,7,8,9,6,...,1,2,0,0,3,1,1,0,0,1
TPT1,12,17,13,19,6,9,16,17,11,10,...,3,0,2,3,1,2,0,3,7,4
RPL13A,48,66,53,57,35,57,56,34,48,50,...,2,6,2,2,7,1,1,6,9,14


### QC of cleaned data
Finally, 90,406 cells were obtained for the downstream analysis. We obtained 1673 genes and 5238 UMIs per cell on average.

###
We opted out the batch effect correction algorithm based on the highly consistent results among patients and the undesirable removal of heterogeneity among cancer cells of individual patients (Fig. S1b). Harmony was used as the batch effect removal method.

### 
We used Seurat 2.3 to first normalize expression matrices by function NormalizeData and ScaleData. Then FindVariable function was applied to select the top 600 variable genes and perform principle component analysis. The first 20 principle components and resolution 1.0 were used with FindClusters function to generate 37 cell clusters.

### 
To assign one of the 11 major cell types to each cluster, we scored each cluster by the normalized expressions of the following canonical markers: Endothelial cells (CLDN5, VWF, PECAM1), Epithelial cells (CAPS, SNTN), Alveolar cells (CLDN18, AQP4, FLOR1), Fibroblasts (COL1A1, COL1A2, DCN), T cells (CD2, CD3D, CD3E, CD3G), B cells (CD79A, CD79B), Myeloid cells (CD14, LYZ), Neutrophils (CSF3R, S100A8, S100A9), Follicular dendritic cells (FDCSP), Mast cells (GATA2, TPSAB1, TPSB2). The highest scored cell type was assigned to each cluster. Cancer cell clusters were negative for normal lung epithelial markers and positive for EPCAM. The clusters assigned to the same cell type were lumped together for the following analysis.

### 
The final results were manually examined to ensure the correctness of the results and visualized by Uniform Manifold Approximation and Projection (UMAP).

###
The 11 major cell types were chosen by initial exploratory inspection of the differentially expressed genes (DEGs) of each cluster combined with literature study. The DEGs were generated by Seurat FindMarkers function.

### 
Wedefined a LUAD and a LUSC score for each patient. The score was calculated based on the average percentage of marker expressions of tumor cells for LUAD (NAPSA and TTF-1) or LUSC (KRT5, DSG, and TP63), and the higher scored subtype was assigned to each patient. If both scores are less than 0.05, translating to 5% of cells expressing given markers, we assigned the patient to NSCLC. Considering both pathological subtype assignment and scRNA subtype assignment (Fig. S2A), we determined a final classification upon the review of experts. All the following grouping was based on the final classification of patients.

### 
We further clustered T cells, B cells, neutrophils, myeloid cells, fibroblasts, endothelial cells, alveolar cells, epithelial cells, and cancer cells individually. We set the resolution to 0.8 for T and B cells. For myeloid cells, endothelial cells and alveolar cells, the resolution was 0.6. For fibroblasts, neutrophils and epithelial cells, we set resolution to 0.4, 0.2, and 1.2, respectively. Within each lineage, we applied an iterative process to remove putative doublet clusters, if any, and reclustered the remaining cells. Putative doublets were identified by double positive expressions of the canonical marker genes of 11 major cell types discussed above. Within T lineage, we used the following markers for subtype identification: CD8+ exhausted T (CD8A, LAG3, and TIGIT), CD8+ effector T(CD8A, GNLY, GZMA, GZMK, GZMB, GZMH), CD4+ naïve T (CCR7, LEF1, IL7R, and SELL), CD4+ Tregs (FOXP3, IL2RA, and IKZF2), CD4+ proliferating (TOP2A, MKI67), and CD4+ Th17-like (KLRB1, RORC). Note that CD4 itself has low RNA expression levels and CD4 T cells were deducted by CD3 positive and CD8 negative. KLRC1, KLRD1, and NKG7 were used as the markers of NK cells. T cell subtypes were also predicted by singleR based on T cell annotations of public dataset GSE992549,36. Similarly, we distinguished follicular B cells (MS4A1, MHCII, CXCR4) from plasma cells (MZB1, JCHAIN, IgH) among the B cell lineage. Plasmacytoid DC (IL3RA, LILRA4, CLEC4C) was clustered in the B cell lineage. For the myeloid clusters, macrophages were positive for canonical marker CD68, and M2-like macrophage markers CD163 and MRC1/CD206. Other myeloid cell types were confirmed by specific marker genes including classical monocytes (CD14, LYZ, VCAN), cDC1 (XCR1, CLEC9A), cDC2 (FCER1A, CD1C), and mature DC (LAMP3). Within fibroblasts (DCN and COL1A1), RGS5 and CSPG4 was used to mark the pericytes. Myofibroblasts were identified by upregulation of ACTA2 and MYH11. For endothelial cells (PECAM1, VWF and ENG), tip cells are characterized by their markers and angiogenesis-related genes (KCNE3, ESM1, ANGPT2, and APLN). Vascular cells mainly consisted of VECs and AECs, respectively identified by their markers ACKR1 and GJA5. A subset of the cells expressing lymphatic markers such as PDPN and PROX1 were defined as LECs-like. Using representative markers for classical airway epithelial cell types, we identified alveolar Type 2 cells (SFTPC, SFTPA1, and ABCA3), club cells (SCGB1A1 and SCGB3A1), basal cells (KRT5, KRT6A, and KRT14), and ciliated cells (FOXJ1, TPPP3, and PIFO) for alveolar cells and other lung epithelial cells. Specific genes for cell-type identification are provided in Supplementary Table 2. For cancer clusters, clustering resolutions 0.2, 0.4, and 0.6 were all used to test the robustness of cell grouping.

###
We applied Monocle2 to determine the lineage differentiation of cell subtypes with potential developmental relationship.

###
For CD4+ T cells, we used Seurat 2.3 FindVariableFeatures function to select top 1500 high variable genes of four CD4 clusters to order cells. DDRTree was used to learn tree-like trajectories. 

### 
The heatmap along the developmental trajectory was only shown for marker genes of T subtypes. For Cancer cells and normal epithelial cells, we used AT2 cells, club cells, and LUAD cancer cell clusters to infer the evolutional paths for LUAD tumors. For LUSC tumors, we selected basal cells, club cells, and LUSC cancer cell clusters. Top 1000 high variable genes were used for both LUAD and LUSC trajectories. We also applied Slingshot to uncover the CD4+ T cell development trajectory. The identified paths were mapped to UMAP projection for visualization.